In [ ]:
!pip install - -upgrade google-cloud-pubsub
!pip install - -upgrade 'apache-beam[gcp]'
!pip install - -upgrade google-apitools

In [ ]:
PROJECT_ID = %env GOOGLE_CLOUD_PROJECT

In [ ]:
import apache_beam as beam
import sys

from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions


sys.argv = ["10_streaming.ipynb"]
encoding = "utf-8"
input_subscription = "projects/{}/subscriptions/movie_sub".format(PROJECT_ID)

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True

p = beam.Pipeline(options=options)

comedy_movies_topic = "projects/{}/topics/comedy_movies".format(PROJECT_ID)

pubsub_pipeline = (
    p
    | "Read from PubSub topic" >> beam.io.ReadFromPubSub(subscription=input_subscription)
    | "Split the records by comma" >> beam.Map(lambda row: row.decode(encoding).split(","))
    | "Filter movies having Comedy as genre" >> beam.Filter(lambda row: "Comedy" in row[2])
    | "Converting to byte String" >> beam.Map(lambda row: "".join(row).encode(encoding))
    | "Write to PubSub" >> beam.io.WriteToPubSub(comedy_movies_topic)
)

result = p.run()
result.wait_until_finish()